In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path

import util
import lines
import group
import label
import records

In [64]:
test_files_dir = ".././test_files/"
test_files = util.list_files(test_files_dir)
file = test_files[5]
display(test_files, file)

['.././test_files/\\Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/\\LS_index_1920.pdf',
 '.././test_files/\\LS_index_1920_split.pdf',
 '.././test_files/\\LS_index_1921.pdf',
 '.././test_files/\\LS_index_1937.pdf',
 '.././test_files/\\LS_index_1944.pdf',
 '.././test_files/\\LS_index_1944_split.pdf',
 '.././test_files/\\LS_index_1976.pdf',
 '.././test_files/\\LS_index_1986.pdf',
 '.././test_files/\\LS_index_1986_split.pdf']

'.././test_files/\\LS_index_1944.pdf'

In [10]:
p_start = 3
pages, dicts = util.read_pdf(file, p_start)

l_df = lines.make_lines_df(dicts)
l_df = lines.merge_close_lines(l_df)
l_df = lines.remove_useless_lines(l_df)

display(l_df)

Reading pdf from .././test_files/\LS_index_1986_split.pdf
...
Finished reading 4 page(s)


,line_text,spans,x0,y0,x1,y1,page
0,1987—List No. 3 6 (112),"[1987—List No. 3 , 6 , (112) ]",75.12,37.47,403.55,51.05,3
1,South Australia (cont.),"[South Australia, (cont.) ]",73.92,63.82,155.66,74.97,3
2,[Regulations govern registration and enrolment...,[[Regulations govern registration and enrolmen...,86.16,75.82,402.21,86.49,3
3,midwives and contain provisions regarding requ...,[midwives and contain provisions regarding req...,85.44,84.22,402.38,94.89,3
4,"Issued pursuant to the Nurses Act 1984, these ...","[Issued pursuant to the Nurses Act 1984, these...",85.44,92.62,402.94,103.29,3
...,...,...,...,...,...,...,...
217,by the employment service for referrals to emp...,[by the employment service for referrals to em...,83.00,551.69,272.05,563.70,6
218,"20/XII/1985 SOR/86-58, P.C. 1985-3757: Unemplo...","[20/XII/1985 SOR/86-58, P.C. 1985-3757: Unempl...",70.80,563.89,399.99,575.90,6
219,under the Unemployment Insurance Act 1971. (ib...,[under the Unemployment Insurance Act 1971. (i...,83.00,572.26,313.74,584.27,6
220,"[An amendment providing, inter alia, that pens...","[[An amendment providing, inter alia, that pen...",83.96,584.94,399.97,596.95,6


In [65]:
pdf_df = util.ocr(file, 1)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
lines_df

Converting pdf pages .././test_files/\LS_index_1944.pdf to images.
Starting OCR...
Done with page 1
Done with page 2
Done with page 3
Done with page 4
Done with page 5
Done with page 6
Done with page 7
Done with page 8
Done with page 9
Done with page 10
Done with page 11
Done with page 12
Done with page 13
Done with page 14
Done with page 15
Done with page 16
Done with page 17
Done with page 18
Done with page 19
Done with page 20
Done with page 21
Done with page 22
Done with page 23
Done with page 24
Done with page 25
Done with page 26
Done with page 27
Done with page 28
Done with page 29
Done with page 30
Done with page 31
Done with page 32
Done with page 33
Done with page 34
Done with page 35
Done with page 36
Done with page 37
Done with page 38
Done with page 39
Done with page 40
Done with page 41
Done with page 42
Done with page 43
Done with page 44
Done with page 45
Done with page 46
Done with page 47
Done with page 48
Done with page 49
Done with page 50
Done with page 51
Done wit

,line_text,x0,y0,x1,y1,page,dx
0,INTERNATIONAL LABOUR OFFICE,144,342,1899,405,1,1755
1,"LEGISLATIVE SERIES, 1944",137,1363,1897,1512,1,1760
2,INDEXES,808,2129,1232,2180,1,424
3,"MONTREAL, CANADA",719,3137,1314,3181,1,595
4,1946,967,3207,1072,3248,1,105
...,...,...,...,...,...,...,...
4184,Apr. 28 L.D. respecting the organisation of la...,345,2947,2056,2991,82,1711
4185,agreements; labour courts]. No. 311. (Zb. pp. ...,608,2993,1881,3039,82,1273
4186,G.O. = Gaceta Oficial de los Estados Unidos de...,351,3081,1649,3126,82,1298
4187,"Zb. = Zbornik sakona 1 naredaba,",345,3138,1087,3177,82,742


In [66]:
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)
display(x0_n, bins_x0, bins_x1)

3

,x0,lines,last_x0,count,page
3,"[144, 137, 149]","[0, 1, 5]",149,3,1
1,719,3,719,1,1
2,808,2,808,1,1
1,"[268, 269, 269]","[20, 21, 22]",269,3,2
6,"[436, 437, 439, 441]","[9, 10, 12, 14]",441,4,2
...,...,...,...,...,...
2,"[125, 129, 132, 133, 133, 134, 136, 136, 137, ...","[4090, 4098, 4102, 4105, 4109, 4112, 4115, 412...",145,14,81
1,"[382, 382, 381, 384, 386, 385, 384, 386, 385, ...","[4091, 4092, 4093, 4095, 4096, 4097, 4099, 410...",394,35,81
0,"[311, 323, 312, 300, 297, 312]","[4172, 4174, 4175, 4179, 4180, 4181]",312,6,82
7,"[306, 303, 313, 320, 316, 327, 313, 332, 331, ...","[4141, 4142, 4143, 4144, 4157, 4159, 4165, 416...",348,16,82


,x1,lines,last_x1,count,page
3,"[1899, 1897, 1901]","[0, 1, 5]",1901,3,1
1,1314,3,1314,1,1
2,1232,2,1232,1,1
0,1072,4,1072,1,1
8,"[1926, 1926, 1928, 1930, 1928]","[8, 9, 12, 14, 15]",1928,5,2
...,...,...,...,...,...
7,1674,4167,1674,1,82
12,1703,4151,1703,1,82
9,1761,4161,1761,1,82
4,1915,4178,1915,1,82


In [67]:
li_df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
l = label.assign_labels(li_df, x0_n)
l.head(20)

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
0,INTERNATIONAL LABOUR OFFICE,144,342,1899,405,1,1755,0,2,other
1,"LEGISLATIVE SERIES, 1944",137,1363,1897,1512,1,1760,0,2,other
2,INDEXES,808,2129,1232,2180,1,424,2,0,end
3,"MONTREAL, CANADA",719,3137,1314,3181,1,595,1,0,start
4,1946,967,3207,1072,3248,1,105,-1,0,other
5,Price: 50 cents (2 s.) April-June 1946,149,3295,1901,3340,1,1752,0,2,other
6,".LEGISLATIVE SERIES, 1944",605,332,1664,411,2,1059,-1,1,other
7,Page,1918,465,2019,508,2,101,-1,1,other
8,1.—List of Laws and Orders of 1944 not publish...,359,568,1926,612,2,1567,-1,2,other
9,"Series for that year which amend, repeal or su...",436,613,1926,656,2,1490,1,2,start


In [68]:
cli_df = correct_x0_types(li_df, bins_x0, bins_x1, x0_n)
ll_df = label.assign_labels(cli_df, x0_n)

lli_df = ll_df.copy()
lli_df["new_label"] = lli_df["label"]
lli_df = label.improve_country_classification(lli_df)

lli_df.head(50)

Width median 1752.7575757575758
[(1658.9333333333332, 2), (1736.6451612903227, 7), (1797.5483870967741, 8), (1739.95625, 9), (1738.5256410256409, 11), (1775.093023255814, 14), (1740.4378378378378, 19), (1734.8974358974358, 23), (1765.8484848484848, 25), (1765.4814814814813, 26), (1764.6410256410256, 32), (1742.65, 37), (1769.9848484848485, 38), (1735.6811594202898, 55), (1735.0833333333333, 61), (1828.6216216216217, 64), (1705.3108974358975, 69), (1707.1991341991343, 70), (1741.2571428571428, 71), (1705.760162601626, 77), (1728.189024390244, 78), (1702.4452690166975, 79), (1739.2894736842106, 81)]
Pages small width [2, 7, 9, 11, 19, 23, 37, 55, 61, 69, 70, 71, 77, 78, 79, 81]
Pages big width [8, 14, 25, 26, 32, 38, 64]


,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label,new_label
0,INTERNATIONAL LABOUR OFFICE,144,342,1899,405,1,1755,0,2,other,other
1,"LEGISLATIVE SERIES, 1944",137,1363,1897,1512,1,1760,0,2,other,other
2,INDEXES,808,2129,1232,2180,1,424,2,0,end,end
3,"MONTREAL, CANADA",719,3137,1314,3181,1,595,1,0,start,start
4,1946,967,3207,1072,3248,1,105,-1,0,other,other
5,Price: 50 cents (2 s.) April-June 1946,149,3295,1901,3340,1,1752,0,2,other,other
6,".LEGISLATIVE SERIES, 1944",605,332,1664,411,2,1059,-1,1,other,other
7,Page,1918,465,2019,508,2,101,-1,1,other,other
8,1.—List of Laws and Orders of 1944 not publish...,359,568,1926,612,2,1567,-1,2,other,other
9,"Series for that year which amend, repeal or su...",436,613,1926,656,2,1490,2,2,middle,middle


In [69]:
rec_gr = records.group_records(lli_df)
rec = records.extract_records(lli_df)
rec.head(20)

,country,region,text,page
0,ARGENTINE REPUBLIC,,"June 3 Decree No. 13,943/44, to repeal section...",3
1,ARGENTINE REPUBLIC,,"Sept. 11 Decrees No. 24,335/44 and 24,336/44, ...",3
2,ARGENTINE REPUBLIC,,"Nov. 4 Decree No. 29,830/44, to amend certain ...",3
3,AUSTRALIA,,"Apr. 6 An Act to,amend the Invalid and Old-age...",3
4,AUSTRALIA,,Dec. 21 Amendment of the National Security (In...,3
5,AUSTRALIA,New South Wales,Dec. 8 An Act to provide for the payment of co...,3
6,AUSTRALIA,New South Wales,B.O. = Boletin Oficial de la Reptblica Argenti...,3
7,AUSTRALIA,New South Wales,"*L.S. 1930: (Arg. 3, B). — ?L.S. 1934 (Arg. 1,...",3
8,AUSTRALIA,New South Wales,*L.S. 1941 (Austral. 1 and 3); 1942 (Austral. ...,3
9,AUSTRALIA,Queensland,Dec. 1 An Act to suspend certain provisions of...,4


In [70]:
rec.to_csv("index_1944.csv")

In [29]:
# correct x0_type for pages where something went wrong
def correct_x0_types(lines_df, bins_x0, bins_x1, x0_n):
    df = lines_df.copy()
    start_page = lines_df["page"].min()

    text_widths = list(lines.make_borders_df(bins_x0, bins_x1)["dx"]) # difference between mean of first and last bin for x0 for every page

    tw = text_widths.copy()
    tw.sort()
    width_median = tw[int(len(tw)/2)]
    print("Width median", width_median)

    f = filter(lambda w: (not util.similar_to(w, width_median, 10)), text_widths) # filter widths that differ from the rest
    f = list(f)

    p = []
    for w in range(len(f)):
        p.append((f[w], text_widths.index(f[w]) + start_page)) # add page to strange width
    
    print(p)

    p_l = [a for w, a in p if w<width_median]
    p_g = [a for w, a in p if w>width_median]
    
    print("Pages small width", p_l)
    print("Pages big width", p_g)

    df.loc[df["page"].isin(p_l) & (df["x0_type"]>=0), "x0_type"] +=1 # correct wrong x0_type for p_l

    # correct wrong x0_type (and x1_type) for p_g
    bins = group.get_line_start_end_bins(df.loc[df["page"].isin(p_g)])
    bins_x0_cor = group.get_relevant_x0_bins(bins[0], x0_n, drop_first=True) # drop bin on the very left
    df_cor = label.assign_types(df.loc[df["page"].isin(p_g)], bins_x0_cor, bins_x1.loc[bins_x1["page"].isin(p_g)], x0_n)
    df.loc[df["page"].isin(p_g), ["x0_type", "x1_type"]] = df_cor[["x0_type", "x1_type"]]

    return df

## Probleme
- Dokumente mit 2 Spalten
- Dokumente mit Länder-Überschrift über ganze Breite der Seite
- Unnötigen Anfangstext auf der ersten Seite rausfiltern
    - 1920

## Fragen
